In [6]:
def parse_input_file(filename):
    with open(filename, "r") as file:
        lines = file.readlines()
    
    # Parse first line (D, R, T)
    D, R, T = map(int, lines[0].split())
    
    # Parse resources
    resources = []
    for i in range(1, R + 1):
        parts = lines[i].split()
        resource = {
            "Resource ID": int(parts[0]), 
            "Activation Cost": int(parts[1]),  # One-time initial expenditure
            "Periodic Cost": int(parts[2]),    # Recurring maintenance cost per turn
            "Active Turns": int(parts[3]),     # Turns the resource stays active
            "Downtime Turns": int(parts[4]),   # Turns needed for maintenance after a cycle
            "Life Cycle": int(parts[5]),       # Total lifespan of the resource
            "Buildings Powered": int(parts[6]),# Number of buildings it supports per active turn
            "Special Effect": parts[7],        # Unique effect or property
            "Efficiency Rating": int(parts[8]) if len(parts) > 8 else None  # Additional performance metric
        }
        resources.append(resource)
    
    # Parse turns
    turns = []
    for i in range(R + 1, R + 1 + T):
        TMt, TXt, TRt = map(int, lines[i].split())
        turns.append({"Minimum Buildings": TMt, "Maximum Buildings": TXt, "Profit": TRt})
    
    return D, R, T, resources, turns

level = "1-thunberg"

D, R, T, resources, turns = parse_input_file(f"{level}.txt")

# Print parsed data
print("Initial Capital:", D)
print("Total Resources:", R)
print("Game Turns:", T)
print("Resources:", resources)
print("Turns:", turns)



Initial Capital: 1000
Total Resources: 15
Game Turns: 50
Resources: [{'Resource ID': 0, 'Activation Cost': 533, 'Periodic Cost': 12, 'Active Turns': 1, 'Downtime Turns': 1, 'Life Cycle': 4, 'Buildings Powered': 2, 'Special Effect': 'X', 'Efficiency Rating': None}, {'Resource ID': 1, 'Activation Cost': 12, 'Periodic Cost': 7, 'Active Turns': 3, 'Downtime Turns': 2, 'Life Cycle': 10, 'Buildings Powered': 1, 'Special Effect': 'A', 'Efficiency Rating': 2}, {'Resource ID': 2, 'Activation Cost': 89, 'Periodic Cost': 36, 'Active Turns': 1, 'Downtime Turns': 0, 'Life Cycle': 2, 'Buildings Powered': 0, 'Special Effect': 'E', 'Efficiency Rating': 11}, {'Resource ID': 3, 'Activation Cost': 238, 'Periodic Cost': 49, 'Active Turns': 3, 'Downtime Turns': 1, 'Life Cycle': 6, 'Buildings Powered': 0, 'Special Effect': 'D', 'Efficiency Rating': 10}, {'Resource ID': 4, 'Activation Cost': 36, 'Periodic Cost': 5, 'Active Turns': 1, 'Downtime Turns': 0, 'Life Cycle': 6, 'Buildings Powered': 1, 'Special Effe

In [ ]:

class Resource:

    current_active_turns = 0
    current_maintenance_turns = 0
    remaining_lyfecycles = 0
    # not in maintenance
    active = False
    # permanently dead
    dead = False
    
    def __init__(self, **kwargs):
        self.resource_id = kwargs["Resource ID"]
        self.activation_cost = kwargs["Activation Cost"]
        self.periodic_cost = kwargs["Periodic Cost"]
        self.active_turns = kwargs["Active Turns"]
        self.downtime_turns = kwargs["Downtime Turns"]
        self.life_cycle = kwargs["Life Cycle"]
        self.buildings_powered = kwargs["Buildings Powered"]
        self.special_effect = kwargs["Special Effect"]
        self.efficiency_rating = kwargs["Efficiency Rating"]

        self.current_active_turns = self.active_turns
        self.remaining_lyfecycles = self.life_cycle
    
    def update(self):
        if self.current_active_turns > 0:
            self.current_active_turns -= 1
            if self.current_active_turns == 0 and self.active:
                self.active = False
                self.current_maintenance_turns = self.downtime_turns
        elif self.current_maintenance_turns > 0:
            self.current_maintenance_turns -= 1
            if self.current_maintenance_turns == 0 and not self.active:
                if self.remaining_lyfecycles > 0:
                    self.active = True
                    self.current_active_turns = self.active_turns
                    self.remaining_lyfecycles -= 1
                else:
                    self.dead = True
class Game:

    def __init__(self, D, R, T, resources, turns):
        self.D = D
        self.R = R
        self.T = T
        self.available_resources = [Resource(**resource) for resource in resources]
        self.turns = turns
        self.current_turn_id = 0
        self.current_budget = D
        self.current_resources = []
    
    def get_currently_active_buildings(self):
        return sum(resource.buildings_powered for resource in self.resources if resource.active)
    
    def perform_turn(self, bought_resources_list: list):
        # periodic costs
        for resource in self.current_resources:
            if resource.dead:
                self.current_resources.remove(resource)
                continue
            self.current_budget -= resource.periodic_cost

        # update resources + activation costs
        for resource in bought_resources_list:
            self.current_budget -= resource.activation_cost
            resource.active = True
            resource.current_active_turns = resource.active_turns
            self.current_resources.append(resource)
        
        # add profits
        current_turn = self.turns[self.current_turn_id]
        max_buildings = current_turn["Maximum Buildings"]
        min_buildings = current_turn["Minimum Buildings"]
        profit = current_turn["Profit"]
        if self.get_currently_active_buildings() >= min_buildings:
            self.current_budget += profit * min(max_buildings, self.get_currently_active_buildings())
        
        for resource in self.current_resources:
            resource.update()
        
        self.current_turn_id += 1

In [25]:
## GAME SEQUENCE

available_resources = resources.copy()
available_budget = D
active_resources = []

for turn in range(T):
    
    resources_to_activate = []
    total_buildings_powered = sum([r["Buildings Powered"] for r in active_resources]) 
    #Get Resource 0's details (Assuming Resource ID 0 is always available)
    resource_0 = next(res for res in resources if res["Special Effect"] == 'A')
    # Calculate how many times we can afford to buy Resource 0 this turn
    max_affordable = available_budget // resource_0["Activation Cost"]
    # Calculate how many we can buy without exceeding the max buildings powered
    max_allowed_by_buildings = (turns[turn]["Maximum Buildings"] - total_buildings_powered) // resource_0["Buildings Powered"]


    # Final number of Resource 0 to buy
    num_to_buy = min(max_affordable, max_allowed_by_buildings)
    
    # Buy and activate the resources
    for _ in range(num_to_buy):
        resources_to_activate.append(resource_0.copy())  # Copy to track its own lifecycle

    
    ##TODO IMPLEMENT THE special effects of everything
    ## Assert whether at any turn we spent more than we have 
    for resource in resources_to_activate:
        available_budget -= resource["Activation Cost"]
        if available_budget < 0:
            # print("Not enough budget")
            break
    for resource in active_resources:
        available_budget -= resource["Periodic Cost"]
        if available_budget < 0:
            # print("Not enough budget")
            break
    
    
    ## add the resources to the active resources
    for resource in resources_to_activate:
        active_resources.append(resource)
        # available_resources.remove(resource)
    # print whether we have more building that we would obtain profit from or if there aren't enough buildings powered
    if sum([resource["Buildings Powered"] for resource in active_resources]) > turns[turn]["Maximum Buildings"]:
        pass   # print("Too many buildings powered")

    if sum([resource["Buildings Powered"] for resource in active_resources]) < turns[turn]["Minimum Buildings"]:
        profit = 0
        # print("Not enough buildings powered")
    else :
        #Calculate profit for each turn and then add it to the available budget
        profit = turns[turn]["Profit"] * min(sum([resource["Buildings Powered"] for resource in active_resources]), turns[turn]["Maximum Buildings"])
        available_budget += profit

    ## Check inside the active resources if any of them expired
    for resource in active_resources:
        resource["Active Turns"] -= 1
        if resource["Active Turns"] == 0:
            active_resources.remove(resource)

    # print the output in the format requested
    print(f"{turn} {len(resources_to_activate)} {' '.join(str(resource['Resource ID']) for resource in resources_to_activate)}")


0 8 1 1 1 1 1 1 1 1
1 5 1 1 1 1 1
2 0 
3 4 1 1 1 1
4 5 1 1 1 1 1
5 2 1 1
6 0 
7 7 1 1 1 1 1 1 1
8 3 1 1 1
9 1 1
10 5 1 1 1 1 1
11 3 1 1 1
12 3 1 1 1
13 3 1 1 1
14 1 1
15 6 1 1 1 1 1 1
16 3 1 1 1
17 2 1 1
18 1 1
19 6 1 1 1 1 1 1
20 0 
21 0 
22 9 1 1 1 1 1 1 1 1 1
23 2 1 1
24 0 
25 6 1 1 1 1 1 1
26 3 1 1 1
27 3 1 1 1
28 4 1 1 1 1
29 0 
30 2 1 1
31 7 1 1 1 1 1 1 1
32 1 1
33 0 
34 1 1
35 0 
36 0 
37 0 
38 0 
39 0 
40 0 
41 0 
42 0 
43 0 
44 0 
45 0 
46 0 
47 0 
48 0 
49 0 
